In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install pyspark 

     |████████████████████████████████| 212.3MB 65kB/s 
     |████████████████████████████████| 204kB 16.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=59e477ac297d93367cd0d8159517bd48aecc616c277a775bcf3dd0a679be7924
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('HR').getOrCreate()
df = spark.read.csv('train_LZdllcl.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- region: string (nullable = true)
 |-- education: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- recruitment_channel: string (nullable = true)
 |-- no_of_trainings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- previous_year_rating: integer (nullable = true)
 |-- length_of_service: integer (nullable = true)
 |-- KPIs_met >80%: integer (nullable = true)
 |-- awards_won?: integer (nullable = true)
 |-- avg_training_score: integer (nullable = true)
 |-- is_promoted: integer (nullable = true)



In [3]:
df.show(10)

+-----------+-----------------+---------+----------------+------+-------------------+---------------+---+--------------------+-----------------+-------------+-----------+------------------+-----------+
|employee_id|       department|   region|       education|gender|recruitment_channel|no_of_trainings|age|previous_year_rating|length_of_service|KPIs_met >80%|awards_won?|avg_training_score|is_promoted|
+-----------+-----------------+---------+----------------+------+-------------------+---------------+---+--------------------+-----------------+-------------+-----------+------------------+-----------+
|      65438|Sales & Marketing| region_7|Master's & above|     f|           sourcing|              1| 35|                   5|                8|            1|          0|                49|          0|
|      65141|       Operations|region_22|      Bachelor's|     m|              other|              1| 30|                   5|                4|            0|          0|                60|   

In [4]:
df=df.drop('employee_id')

In [5]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
department,Sales & Marketing,Operations,Sales & Marketing,Sales & Marketing,Technology
region,region_7,region_22,region_19,region_23,region_26
education,Master's & above,Bachelor's,Bachelor's,Bachelor's,Bachelor's
gender,f,m,m,m,m
recruitment_channel,sourcing,other,sourcing,other,other
no_of_trainings,1,1,1,2,1
age,35,30,34,39,45
previous_year_rating,5,5,3,1,3
length_of_service,8,4,7,10,2
KPIs_met >80%,1,0,0,0,0


In [6]:
categorical_feautres=['department','region','education','gender','recruitment_channel']

numerical_cat_feautres=['no_of_trainings','age','previous_year_rating','KPIs_met >80%','awards_won?']

numerical_feautres=['length_of_service','avg_training_score']

Target=['is_promoted']

In [7]:
df.columns

['department',
 'region',
 'education',
 'gender',
 'recruitment_channel',
 'no_of_trainings',
 'age',
 'previous_year_rating',
 'length_of_service',
 'KPIs_met >80%',
 'awards_won?',
 'avg_training_score',
 'is_promoted']

In [8]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
department,54808,None,None,Analytics,Technology
region,54808,None,None,region_1,region_9
education,52399,None,None,Bachelor's,Master's & above
gender,54808,None,None,f,m
recruitment_channel,54808,None,None,other,sourcing
no_of_trainings,54808,1.253010509414684,0.6092640176487848,1,10
age,54808,34.80391548679025,7.660169201585432,20,60
previous_year_rating,50684,3.329255780917055,1.259992551940845,1,5
length_of_service,54808,5.865512333965844,4.2650941796858435,1,37


In [9]:
df.toPandas()[df.columns].nunique()

department               9
region                  34
education                3
gender                   2
recruitment_channel      3
no_of_trainings         10
age                     41
previous_year_rating     5
length_of_service       35
KPIs_met >80%            2
awards_won?              2
avg_training_score      61
is_promoted              2
dtype: int64

In [10]:
categorical_feautres=['department','region','education','gender','recruitment_channel']

numerical_cat_feautres=['no_of_trainings','age','previous_year_rating','KPIs_met >80%','awards_won?']

numerical_feautres=['length_of_service','avg_training_score']

Target=['is_promoted']

In [11]:
df.toPandas().isna().sum()

department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [12]:
df.filter(df['education'].isNull()).count()  , df.filter(df['previous_year_rating'].isNull()).count()

(2409, 4124)

In [13]:
mode_edu=df.groupby("education").count().orderBy("count", ascending=False).first()[0]
mode_pyr=df.groupby("previous_year_rating").count().orderBy("count", ascending=False).first()[0]


In [14]:
mode_edu,mode_pyr

("Bachelor's", 3)

In [15]:
df=df.na.fill(value="Bachelor's",subset=["education"])
df=df.na.fill(value=3,subset=["previous_year_rating"])

In [ ]:
import pyspark.sql.utils
try: 
   for j in df_final.toPandas().columns:
       print("Correlation with "+j+"  is_promoted",df_final.corr('is_promoted',j))
except pyspark.sql.utils.AnalysisException:
      print("Unable to process your query dude!!")   

In [17]:
stages = []
from pyspark.ml.feature import StringIndexer,OneHotEncoder,VectorAssembler
categorical_feautres=['department','region','education','gender','recruitment_channel']
for categoricalCol in categorical_feautres:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

label_stringIdx = StringIndexer(inputCol = 'is_promoted', outputCol = 'label')
stages += [label_stringIdx]

assemblerInputs = [c + "classVec" for c in categorical_feautres] + numerical_feautres + numerical_cat_feautres

assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [18]:
cols=df.columns

In [19]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df_vec = pipelineModel.transform(df)
selectedCols = ['label', 'features'] + cols
df_vec = df_vec.select(selectedCols)
df_vec.printSchema()



root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- department: string (nullable = true)
 |-- region: string (nullable = true)
 |-- education: string (nullable = false)
 |-- gender: string (nullable = true)
 |-- recruitment_channel: string (nullable = true)
 |-- no_of_trainings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- previous_year_rating: integer (nullable = true)
 |-- length_of_service: integer (nullable = true)
 |-- KPIs_met >80%: integer (nullable = true)
 |-- awards_won?: integer (nullable = true)
 |-- avg_training_score: integer (nullable = true)
 |-- is_promoted: integer (nullable = true)



In [20]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
department,Sales & Marketing,Operations,Sales & Marketing,Sales & Marketing,Technology
region,region_7,region_22,region_19,region_23,region_26
education,Master's & above,Bachelor's,Bachelor's,Bachelor's,Bachelor's
gender,f,m,m,m,m
recruitment_channel,sourcing,other,sourcing,other,other
no_of_trainings,1,1,1,2,1
age,35,30,34,39,45
previous_year_rating,5,5,3,1,3
length_of_service,8,4,7,10,2
KPIs_met >80%,1,0,0,0,0


In [23]:
train, test = df_vec.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 38407
Test Dataset Count: 16401


In [26]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label',maxDepth=5, impurity='gini')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.select( 'label', 'rawPrediction', 'prediction', 'probability').show(10)

+-----+--------------------+----------+--------------------+
|label|       rawPrediction|prediction|         probability|
+-----+--------------------+----------+--------------------+
|  0.0|[18.8514773708260...|       0.0|[0.94257386854130...|
|  0.0|[18.8693954148972...|       0.0|[0.94346977074486...|
|  0.0|[18.8693954148972...|       0.0|[0.94346977074486...|
|  0.0|[18.8514773708260...|       0.0|[0.94257386854130...|
|  0.0|[18.8514773708260...|       0.0|[0.94257386854130...|
|  0.0|[18.8514773708260...|       0.0|[0.94257386854130...|
|  0.0|[18.8693954148972...|       0.0|[0.94346977074486...|
|  0.0|[18.8514773708260...|       0.0|[0.94257386854130...|
|  0.0|[18.8514773708260...|       0.0|[0.94257386854130...|
|  0.0|[18.8514773708260...|       0.0|[0.94257386854130...|
+-----+--------------------+----------+--------------------+
only showing top 10 rows



In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.8351257649425928


In [29]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(maxIter=10,)
gbtModel = gbt.fit(train)
predictions = gbtModel.transform(test)
predictions.select( 'label', 'rawPrediction', 'prediction', 'probability').show(10)

+-----+--------------------+----------+--------------------+
|label|       rawPrediction|prediction|         probability|
+-----+--------------------+----------+--------------------+
|  0.0|[1.32033491990410...|       0.0|[0.93343359721933...|
|  0.0|[1.32033491990410...|       0.0|[0.93343359721933...|
|  0.0|[1.32033491990410...|       0.0|[0.93343359721933...|
|  0.0|[1.32033491990410...|       0.0|[0.93343359721933...|
|  0.0|[1.32033491990410...|       0.0|[0.93343359721933...|
|  0.0|[1.32033491990410...|       0.0|[0.93343359721933...|
|  0.0|[1.32033491990410...|       0.0|[0.93343359721933...|
|  0.0|[1.32033491990410...|       0.0|[0.93343359721933...|
|  0.0|[1.32033491990410...|       0.0|[0.93343359721933...|
|  0.0|[1.32033491990410...|       0.0|[0.93343359721933...|
+-----+--------------------+----------+--------------------+
only showing top 10 rows



In [40]:
gbtModel.featureImportances

SparseVector(53, {0: 0.227, 1: 0.1039, 2: 0.0479, 3: 0.0083, 4: 0.0128, 5: 0.0456, 6: 0.0214, 8: 0.0004, 9: 0.0006, 10: 0.0029, 15: 0.0033, 16: 0.0003, 20: 0.0021, 21: 0.0013, 23: 0.0001, 25: 0.0008, 26: 0.001, 27: 0.0006, 31: 0.0004, 41: 0.0002, 43: 0.0014, 44: 0.0004, 46: 0.017, 47: 0.2983, 48: 0.0021, 49: 0.0134, 50: 0.0232, 51: 0.1229, 52: 0.0405})

In [30]:
print(gbt.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 

In [31]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.8955473567961648


In [38]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [2, 4, 6])
             .addGrid(gbt.maxBins, [20, 60])
             .addGrid(gbt.maxIter, [10, 20])
             .build())
cv = CrossValidator(estimator=gbt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
cvModel = cv.fit(train)
predictions = cvModel.transform(test)
evaluator.evaluate(predictions)

0.9052997769459541